In [180]:
%run paths.ipynb
%run preprocess.ipynb

In [181]:
p = parseNeuralSum()

In [182]:
p.parse_folder_1(input_folder=CONFIG.RAW_CNN_TRAIN_FOLDER, target_document_folder=CONFIG.CNN_TRAIN_DOCUMENTS_FOLDER, 
                 target_output_folder=CONFIG.CNN_TRAIN_OUTPUT_FOLDER, target_summary_folder=CONFIG.CNN_TRAIN_SUMMARY_FOLDER)
p.parse_folder_1(input_folder=CONFIG.RAW_CNN_TEST_FOLDER, target_document_folder=CONFIG.CNN_TEST_DOCUMENTS_FOLDER, 
                 target_output_folder=CONFIG.CNN_TEST_OUTPUT_FOLDER, target_summary_folder=CONFIG.CNN_TEST_SUMMARY_FOLDER)
p.parse_folder_1(input_folder=CONFIG.RAW_CNN_VALIDATION_FOLDER, target_document_folder=CONFIG.CNN_VALIDATION_DOCUMENTS_FOLDER, 
                 target_output_folder=CONFIG.CNN_VALIDATION_OUTPUT_FOLDER, target_summary_folder=CONFIG.CNN_VALIDATION_SUMMARY_FOLDER)

In [124]:
g = NeuralSumGlove(glove_filename=CONFIG.GLOVE_FILENAME)

In [125]:
g.convert_to_indices_(document_folder=CONFIG.CNN_TRAIN_DOCUMENTS_FOLDER, output_folder=CONFIG.CNN_TRAIN_OUTPUT_FOLDER, 
                     dump_file=CONFIG.CNN_TRAIN_DUMP)
g.convert_to_indices_(document_folder=CONFIG.CNN_TEST_DOCUMENTS_FOLDER, output_folder=CONFIG.CNN_TEST_OUTPUT_FOLDER, 
                     dump_file=CONFIG.CNN_TEST_DUMP)
g.convert_to_indices_(document_folder=CONFIG.CNN_VALIDATION_DOCUMENTS_FOLDER, output_folder=CONFIG.CNN_VALIDATION_OUTPUT_FOLDER, 
                     dump_file=CONFIG.CNN_VALIDATION_DUMP)

In [126]:
g.dump_all(CONFIG.GLOVE_CACHE_FILENAME)

In [127]:
d = NeuralSumDataHandler()

In [128]:
d.load_dump(CONFIG.CNN_TRAIN_DUMP)
d.pad(output=True)
d.padded_dump(CONFIG.CNN_PADDED_TRAIN_DUMP)
d.load_dump(CONFIG.CNN_TEST_DUMP)
d.pad(output=True)
d.padded_dump(CONFIG.CNN_PADDED_TEST_DUMP)

**RUN FROM HERE FOR TRAINING**

In [129]:
%run paths.ipynb
%run preprocess.ipynb
%run model.ipynb
%run utils.ipynb

In [130]:
d = NeuralSumDataHandler()
d.load_padded_dump(CONFIG.CNN_PADDED_TRAIN_DUMP)
d.make_batches(20)
g = NeuralSumGlove(dump_filename=CONFIG.GLOVE_CACHE_FILENAME)
torch.cuda.set_device(0)

In [6]:
m = EncoderDecoder(g.vectors, word_emb_size=g.dim, sen_emb_size=150, doc_emb_size=200, sen_len=50, batch_size=20, output_dim=1)
m.cuda()

EncoderDecoder(
  (sentenceEncoder): LSTMSentenceEncoderParallel(
    (embeddings): Embedding(400001, 100, padding_idx=400000)
    (sentenceEncoder): LSTM(100, 150, batch_first=True)
  )
  (documentEncoder): LSTM(150, 200, batch_first=True)
  (documentDecoder): LSTM(150, 200, batch_first=True)
  (classifier): Linear(in_features=200, out_features=1, bias=True)
)

In [ ]:
train_classifier(model=m, lines=d.lines, output=d.output, batches=d.batches, iterations=10)
torch.save(m.state_dict(), 'parameters')

**RUN FROM HERE FOR TESTING**

In [45]:
%run paths.ipynb
%run preprocess.ipynb
%run model.ipynb
%run utils.ipynb

In [35]:
d = NeuralSumDataHandler()
d.load_padded_dump(CONFIG.CNN_PADDED_TEST_DUMP)
d.make_batches(20)
g = NeuralSumGlove(dump_filename=CONFIG.GLOVE_CACHE_FILENAME)

In [36]:
m = EncoderDecoder(g.vectors, word_emb_size=g.dim, sen_emb_size=150, doc_emb_size=200, sen_len=50, batch_size=20, output_dim=1)
m.load_state_dict(torch.load('parameters'))
m.cuda()

EncoderDecoder(
  (sentenceEncoder): LSTMSentenceEncoderParallel(
    (embeddings): Embedding(400001, 100, padding_idx=400000)
    (sentenceEncoder): LSTM(100, 150, batch_first=True)
  )
  (documentEncoder): LSTM(150, 200, batch_first=True)
  (documentDecoder): LSTM(150, 200, batch_first=True)
  (classifier): Linear(in_features=200, out_features=1, bias=True)
)

In [46]:
get_summaries(m, d.lines, d.batches, doc_folder=CONFIG.CNN_TEST_DOCUMENTS_FOLDER, write_folder=CONFIG.PREDICTED_SUMMARY_FOLDER)
output = computeRouge(CONFIG.PREDICTED_SUMMARY_FOLDER, CONFIG.CNN_TEST_SUMMARY_FOLDER)

2018-11-27 08:20:20,526 [MainThread  ] [INFO ]  Writing summaries.
2018-11-27 08:20:20,527 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmp_p3kqk6y/system and model files to /tmp/tmp_p3kqk6y/model.
2018-11-27 08:20:20,527 [MainThread  ] [INFO ]  Processing files in /home/ramkishore.s/refresh/data/summaries/.
2018-11-27 08:20:20,528 [MainThread  ] [INFO ]  Processing 834.
2018-11-27 08:20:20,528 [MainThread  ] [INFO ]  Processing 1044.
2018-11-27 08:20:20,529 [MainThread  ] [INFO ]  Processing 717.
2018-11-27 08:20:20,530 [MainThread  ] [INFO ]  Processing 29.
2018-11-27 08:20:20,530 [MainThread  ] [INFO ]  Processing 506.
2018-11-27 08:20:20,531 [MainThread  ] [INFO ]  Processing 303.
2018-11-27 08:20:20,532 [MainThread  ] [INFO ]  Processing 819.
2018-11-27 08:20:20,532 [MainThread  ] [INFO ]  Processing 728.
2018-11-27 08:20:20,533 [MainThread  ] [INFO ]  Processing 1038.
2018-11-27 08:20:20,533 [MainThread  ] [INFO ]  Processing 978.
2018-11-27 08:20:20,

2018-11-27 08:20:20,597 [MainThread  ] [INFO ]  Processing 344.
2018-11-27 08:20:20,598 [MainThread  ] [INFO ]  Processing 877.
2018-11-27 08:20:20,598 [MainThread  ] [INFO ]  Processing 712.
2018-11-27 08:20:20,599 [MainThread  ] [INFO ]  Processing 909.
2018-11-27 08:20:20,599 [MainThread  ] [INFO ]  Processing 907.
2018-11-27 08:20:20,600 [MainThread  ] [INFO ]  Processing 833.
2018-11-27 08:20:20,600 [MainThread  ] [INFO ]  Processing 1079.
2018-11-27 08:20:20,601 [MainThread  ] [INFO ]  Processing 55.
2018-11-27 08:20:20,602 [MainThread  ] [INFO ]  Processing 778.
2018-11-27 08:20:20,602 [MainThread  ] [INFO ]  Processing 136.
2018-11-27 08:20:20,603 [MainThread  ] [INFO ]  Processing 919.
2018-11-27 08:20:20,603 [MainThread  ] [INFO ]  Processing 146.
2018-11-27 08:20:20,604 [MainThread  ] [INFO ]  Processing 537.
2018-11-27 08:20:20,604 [MainThread  ] [INFO ]  Processing 401.
2018-11-27 08:20:20,605 [MainThread  ] [INFO ]  Processing 955.
2018-11-27 08:20:20,605 [MainThread  ] [

2018-11-27 08:20:20,669 [MainThread  ] [INFO ]  Processing 76.
2018-11-27 08:20:20,669 [MainThread  ] [INFO ]  Processing 147.
2018-11-27 08:20:20,670 [MainThread  ] [INFO ]  Processing 41.
2018-11-27 08:20:20,670 [MainThread  ] [INFO ]  Processing 1000.
2018-11-27 08:20:20,671 [MainThread  ] [INFO ]  Processing 465.
2018-11-27 08:20:20,671 [MainThread  ] [INFO ]  Processing 211.
2018-11-27 08:20:20,672 [MainThread  ] [INFO ]  Processing 655.
2018-11-27 08:20:20,672 [MainThread  ] [INFO ]  Processing 896.
2018-11-27 08:20:20,673 [MainThread  ] [INFO ]  Processing 619.
2018-11-27 08:20:20,674 [MainThread  ] [INFO ]  Processing 1048.
2018-11-27 08:20:20,674 [MainThread  ] [INFO ]  Processing 616.
2018-11-27 08:20:20,675 [MainThread  ] [INFO ]  Processing 970.
2018-11-27 08:20:20,675 [MainThread  ] [INFO ]  Processing 475.
2018-11-27 08:20:20,676 [MainThread  ] [INFO ]  Processing 39.
2018-11-27 08:20:20,676 [MainThread  ] [INFO ]  Processing 12.
2018-11-27 08:20:20,677 [MainThread  ] [IN

2018-11-27 08:20:20,740 [MainThread  ] [INFO ]  Processing 812.
2018-11-27 08:20:20,741 [MainThread  ] [INFO ]  Processing 923.
2018-11-27 08:20:20,741 [MainThread  ] [INFO ]  Processing 569.
2018-11-27 08:20:20,742 [MainThread  ] [INFO ]  Processing 571.
2018-11-27 08:20:20,742 [MainThread  ] [INFO ]  Processing 592.
2018-11-27 08:20:20,743 [MainThread  ] [INFO ]  Processing 801.
2018-11-27 08:20:20,744 [MainThread  ] [INFO ]  Processing 1033.
2018-11-27 08:20:20,744 [MainThread  ] [INFO ]  Processing 184.
2018-11-27 08:20:20,745 [MainThread  ] [INFO ]  Processing 947.
2018-11-27 08:20:20,745 [MainThread  ] [INFO ]  Processing 306.
2018-11-27 08:20:20,746 [MainThread  ] [INFO ]  Processing 393.
2018-11-27 08:20:20,746 [MainThread  ] [INFO ]  Processing 153.
2018-11-27 08:20:20,747 [MainThread  ] [INFO ]  Processing 152.
2018-11-27 08:20:20,747 [MainThread  ] [INFO ]  Processing 276.
2018-11-27 08:20:20,748 [MainThread  ] [INFO ]  Processing 51.
2018-11-27 08:20:20,749 [MainThread  ] [

2018-11-27 08:20:20,813 [MainThread  ] [INFO ]  Processing 1070.
2018-11-27 08:20:20,813 [MainThread  ] [INFO ]  Processing 490.
2018-11-27 08:20:20,814 [MainThread  ] [INFO ]  Processing 683.
2018-11-27 08:20:20,814 [MainThread  ] [INFO ]  Processing 75.
2018-11-27 08:20:20,815 [MainThread  ] [INFO ]  Processing 26.
2018-11-27 08:20:20,816 [MainThread  ] [INFO ]  Processing 788.
2018-11-27 08:20:20,816 [MainThread  ] [INFO ]  Processing 1063.
2018-11-27 08:20:20,817 [MainThread  ] [INFO ]  Processing 739.
2018-11-27 08:20:20,817 [MainThread  ] [INFO ]  Processing 1039.
2018-11-27 08:20:20,818 [MainThread  ] [INFO ]  Processing 43.
2018-11-27 08:20:20,818 [MainThread  ] [INFO ]  Processing 382.
2018-11-27 08:20:20,819 [MainThread  ] [INFO ]  Processing 515.
2018-11-27 08:20:20,819 [MainThread  ] [INFO ]  Processing 715.
2018-11-27 08:20:20,820 [MainThread  ] [INFO ]  Processing 974.
2018-11-27 08:20:20,821 [MainThread  ] [INFO ]  Processing 428.
2018-11-27 08:20:20,821 [MainThread  ] [

...

2018-11-27 08:20:21,468 [MainThread  ] [INFO ]  Processing 326.
2018-11-27 08:20:21,468 [MainThread  ] [INFO ]  Processing 781.
2018-11-27 08:20:21,469 [MainThread  ] [INFO ]  Processing 437.
2018-11-27 08:20:21,469 [MainThread  ] [INFO ]  Processing 1020.
2018-11-27 08:20:21,470 [MainThread  ] [INFO ]  Processing 561.
2018-11-27 08:20:21,470 [MainThread  ] [INFO ]  Processing 435.
2018-11-27 08:20:21,471 [MainThread  ] [INFO ]  Processing 680.
2018-11-27 08:20:21,472 [MainThread  ] [INFO ]  Processing 1034.
2018-11-27 08:20:21,472 [MainThread  ] [INFO ]  Processing 187.
2018-11-27 08:20:21,473 [MainThread  ] [INFO ]  Processing 977.
2018-11-27 08:20:21,473 [MainThread  ] [INFO ]  Processing 668.
2018-11-27 08:20:21,474 [MainThread  ] [INFO ]  Processing 894.
2018-11-27 08:20:21,474 [MainThread  ] [INFO ]  Processing 148.
2018-11-27 08:20:21,475 [MainThread  ] [INFO ]  Processing 199.
2018-11-27 08:20:21,476 [MainThread  ] [INFO ]  Processing 818.
2018-11-27 08:20:21,476 [MainThread  ]

2018-11-27 08:20:21,540 [MainThread  ] [INFO ]  Processing 667.
2018-11-27 08:20:21,541 [MainThread  ] [INFO ]  Processing 353.
2018-11-27 08:20:21,542 [MainThread  ] [INFO ]  Processing 1053.
2018-11-27 08:20:21,542 [MainThread  ] [INFO ]  Processing 180.
2018-11-27 08:20:21,543 [MainThread  ] [INFO ]  Processing 101.
2018-11-27 08:20:21,543 [MainThread  ] [INFO ]  Processing 449.
2018-11-27 08:20:21,544 [MainThread  ] [INFO ]  Processing 693.
2018-11-27 08:20:21,544 [MainThread  ] [INFO ]  Processing 272.
2018-11-27 08:20:21,545 [MainThread  ] [INFO ]  Processing 193.
2018-11-27 08:20:21,546 [MainThread  ] [INFO ]  Processing 432.
2018-11-27 08:20:21,546 [MainThread  ] [INFO ]  Processing 694.
2018-11-27 08:20:21,547 [MainThread  ] [INFO ]  Processing 630.
2018-11-27 08:20:21,547 [MainThread  ] [INFO ]  Processing 599.
2018-11-27 08:20:21,548 [MainThread  ] [INFO ]  Processing 165.
2018-11-27 08:20:21,548 [MainThread  ] [INFO ]  Processing 10.
2018-11-27 08:20:21,549 [MainThread  ] [

2018-11-27 08:20:21,613 [MainThread  ] [INFO ]  Processing 963.
2018-11-27 08:20:21,614 [MainThread  ] [INFO ]  Processing 988.
2018-11-27 08:20:21,615 [MainThread  ] [INFO ]  Processing 895.
2018-11-27 08:20:21,615 [MainThread  ] [INFO ]  Processing 944.
2018-11-27 08:20:21,616 [MainThread  ] [INFO ]  Processing 847.
2018-11-27 08:20:21,616 [MainThread  ] [INFO ]  Processing 347.
2018-11-27 08:20:21,617 [MainThread  ] [INFO ]  Processing 504.
2018-11-27 08:20:21,617 [MainThread  ] [INFO ]  Processing 169.
2018-11-27 08:20:21,618 [MainThread  ] [INFO ]  Processing 890.
2018-11-27 08:20:21,619 [MainThread  ] [INFO ]  Processing 88.
2018-11-27 08:20:21,619 [MainThread  ] [INFO ]  Processing 361.
2018-11-27 08:20:21,620 [MainThread  ] [INFO ]  Processing 1035.
2018-11-27 08:20:21,620 [MainThread  ] [INFO ]  Processing 698.
2018-11-27 08:20:21,621 [MainThread  ] [INFO ]  Processing 405.
2018-11-27 08:20:21,621 [MainThread  ] [INFO ]  Processing 201.
2018-11-27 08:20:21,622 [MainThread  ] [

2018-11-27 08:20:21,687 [MainThread  ] [INFO ]  Processing 673.
2018-11-27 08:20:21,688 [MainThread  ] [INFO ]  Processing 985.
2018-11-27 08:20:21,688 [MainThread  ] [INFO ]  Processing 642.
2018-11-27 08:20:21,689 [MainThread  ] [INFO ]  Processing 699.
2018-11-27 08:20:21,690 [MainThread  ] [INFO ]  Processing 427.
2018-11-27 08:20:21,690 [MainThread  ] [INFO ]  Processing 98.
2018-11-27 08:20:21,691 [MainThread  ] [INFO ]  Processing 426.
2018-11-27 08:20:21,691 [MainThread  ] [INFO ]  Processing 92.
2018-11-27 08:20:21,692 [MainThread  ] [INFO ]  Processing 512.
2018-11-27 08:20:21,692 [MainThread  ] [INFO ]  Processing 83.
2018-11-27 08:20:21,693 [MainThread  ] [INFO ]  Processing 1009.
2018-11-27 08:20:21,694 [MainThread  ] [INFO ]  Processing 228.
2018-11-27 08:20:21,694 [MainThread  ] [INFO ]  Processing 773.
2018-11-27 08:20:21,695 [MainThread  ] [INFO ]  Processing 551.
2018-11-27 08:20:21,695 [MainThread  ] [INFO ]  Processing 387.
2018-11-27 08:20:21,696 [MainThread  ] [IN

2018-11-27 08:20:21,761 [MainThread  ] [INFO ]  Processing 891.
2018-11-27 08:20:21,762 [MainThread  ] [INFO ]  Processing 412.
2018-11-27 08:20:21,762 [MainThread  ] [INFO ]  Processing 262.
2018-11-27 08:20:21,763 [MainThread  ] [INFO ]  Processing 854.
2018-11-27 08:20:21,763 [MainThread  ] [INFO ]  Processing 679.
2018-11-27 08:20:21,764 [MainThread  ] [INFO ]  Processing 922.
2018-11-27 08:20:21,764 [MainThread  ] [INFO ]  Processing 282.
2018-11-27 08:20:21,765 [MainThread  ] [INFO ]  Processing 687.
2018-11-27 08:20:21,766 [MainThread  ] [INFO ]  Processing 331.
2018-11-27 08:20:21,766 [MainThread  ] [INFO ]  Saved processed files to /tmp/tmp_p3kqk6y/model.
2018-11-27 08:20:22,585 [MainThread  ] [INFO ]  Written ROUGE configuration to /tmp/tmpctdax5m5/rouge_conf.xml
2018-11-27 08:20:22,585 [MainThread  ] [INFO ]  Running ROUGE with command /home/ramkishore.s/ROUGE-1.5.5/ROUGE-1.5.5.pl -e /home/ramkishore.s/ROUGE-1.5.5/data -c 95 -2 -1 -U -r 1000 -n 4 -w 1.2 -a -m /tmp/tmpctdax5m

In [47]:
# r1 fscore r2 fscore rl f score
# state of the art is around: 30, 10, 24
output

(0.25265, 0.08759, 0.22418)

In [38]:
get_file(1001, CONFIG.PREDICTED_SUMMARY_FOLDER)

tensor([0, 2, 5], device='cuda:0')
@entity1 , @entity2 amid growing scrutiny over whether a 73 - year - old volunteer deputy who killed a suspect during a sting operation was qualified to be policing the streets , a new report raises a troubling allegation
claims that the volunteer deputy 's records had been falsified emerged " almost immediately " from multiple sources after @entity15 killed @entity23 on april 2 , reporter @entity18 said
but the orders apparently started years ago , before @entity23 ' death , " back when ( @entity15 ) was trying to get on as a deputy , " reporter @entity31 told @entity3 's " @entity35



In [39]:
get_file(1001, CONFIG.CNN_TEST_SUMMARY_FOLDER)

@entity153 in @entity154 says @entity15 never trained with them
" he met every requirement , and all he did was give of himself , " his attorney says
@entity11 newspaper : three supervisors who refused to sign forged records on @entity15 were reassigned

